In [3]:
import os

import datetime

import numpy as np
import pandas as pd

In [6]:
data_folder = os.path.join("../../","datasets","initial_data_split")

train = pd.read_csv(os.path.join(data_folder,"train.csv"))
print train.head(5)

              datetime  season  holiday  workingday  weather  temp   atemp  \
0  2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1  2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2  2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3  2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4  2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40  
2        80        0.0       5          27     32  
3        75        0.0       3          10     13  
4        75        0.0       0           1      1  


In [40]:
example_datetime = train[["datetime"]].values[10][0]
print example_datetime

2011-01-01 10:00:00


In [41]:
example_datetime = datetime.datetime.strptime(example_datetime, "%Y-%m-%d %H:%M:%S")

In [42]:
print example_datetime.weekday()
print example_datetime.year
print example_datetime.month
print example_datetime.hour

5
2011
1
10


In [53]:
new_train = train
new_train['datetime'] =  pd.to_datetime(new_train['datetime'], format="%Y-%m-%d %H:%M:%S")
new_train['time_hour'] = new_train['datetime'].apply(lambda x: x.hour)
new_train['date_weekday'] = new_train['datetime'].apply(lambda x: x.weekday())
new_train['date_month'] = new_train['datetime'].apply(lambda x: x.month)
new_train['date_year'] = new_train['datetime'].apply(lambda x: x.year)

In [54]:
print new_train.head(5)

             datetime  season  holiday  workingday  weather  temp   atemp  \
0 2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1 2011-01-01 01:00:00       1        0           0        1  9.02  13.635   
2 2011-01-01 02:00:00       1        0           0        1  9.02  13.635   
3 2011-01-01 03:00:00       1        0           0        1  9.84  14.395   
4 2011-01-01 04:00:00       1        0           0        1  9.84  14.395   

   humidity  windspeed  casual  registered  count  time_hour  date_weekday  \
0        81        0.0       3          13     16          0             5   
1        80        0.0       8          32     40          1             5   
2        80        0.0       5          27     32          2             5   
3        75        0.0       3          10     13          3             5   
4        75        0.0       0           1      1          4             5   

   date_month  date_year  
0           1       2011  
1           1 